**RESNET With early stopping, batch normalaization, drop out**

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

# Data transformations
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the dataset
dataset = ImageFolder(root='/kaggle/input/covidct', transform=train_transform)

# Define your custom CNN model by fine-tuning ResNet-18
class CustomResNet(nn.Module):
    def __init__(self, dropout_prob=0.5):
        super(CustomResNet, self).__init__()
        # Load a pre-trained ResNet-18 model
        self.resnet = models.resnet18(pretrained=True)
        
        # Remove the last classification layer
        num_ftrs = self.resnet.fc.in_features
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-1])
        
        # Add a new classification layer
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_ftrs, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(dropout_prob),
            nn.Linear(512, 2)
        )

    def forward(self, x):
        x = self.resnet(x)
        x = self.fc(x)
        return x

# Define hyperparameters
learning_rate = 0.001
num_epochs = 10
batch_size = 32

# Define 5-fold cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

average_precision = 0
average_recall = 0
average_f1 = 0

for fold, (train_indices, val_indices) in enumerate(kf.split(dataset, dataset.targets)):
    train_set = torch.utils.data.Subset(dataset, train_indices)
    val_set = torch.utils.data.Subset(dataset, val_indices)
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=batch_size)

    # Create and initialize your custom ResNet model
    model = CustomResNet(dropout_prob=0.5)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Implement early stopping
    best_val_loss = float('inf')
    patience = 3
    no_improvement = 0

    for epoch in range(num_epochs):
        for batch_idx, (images, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        val_loss = 0.0
        model.eval()
        with torch.no_grad():
            for images, labels in val_loader:
                outputs = model(images)
                val_loss += criterion(outputs, labels)
        
        val_loss /= len(val_loader)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            no_improvement = 0
            # Save the best model checkpoint
            torch.save(model.state_dict(), f"fold_{fold}_best_model.pth")
        else:
            no_improvement += 1
            if no_improvement >= patience:
                print(f"Early stopping after {epoch+1} epochs.")
                break

    # Evaluate on the validation set
    model.load_state_dict(torch.load(f"fold_{fold}_best_model.pth"))
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.tolist())
            all_labels.extend(labels.tolist())

    # Calculate and record precision, recall, and F1 scores for this fold
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)

    print(f"Fold {fold+1} - Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

    average_precision += precision
    average_recall += recall
    average_f1 += f1

# Calculate average precision, recall, and F1 scores across all folds
average_precision /= 5
average_recall /= 5
average_f1 /= 5

print(f"Average Precision: {average_precision:.4f}")
print(f"Average Recall: {average_recall:.4f}")
print(f"Average F1 Score: {average_f1:.4f}")


c:\Users\KOBINATH\.conda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\KOBINATH\.conda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Early stopping after 8 epochs.
Fold 1 - Precision: 0.5417, Recall: 0.9750, F1 Score: 0.6964


c:\Users\KOBINATH\.conda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\KOBINATH\.conda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Early stopping after 7 epochs.
Fold 2 - Precision: 0.5302, Recall: 1.0000, F1 Score: 0.6930


c:\Users\KOBINATH\.conda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\KOBINATH\.conda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Early stopping after 8 epochs.
Fold 3 - Precision: 0.5302, Recall: 1.0000, F1 Score: 0.6930


c:\Users\KOBINATH\.conda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\KOBINATH\.conda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Early stopping after 5 epochs.
Fold 4 - Precision: 0.5302, Recall: 1.0000, F1 Score: 0.6930


c:\Users\KOBINATH\.conda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\KOBINATH\.conda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Early stopping after 8 epochs.
Fold 5 - Precision: 0.5369, Recall: 1.0000, F1 Score: 0.6987
Average Precision: 0.5338
Average Recall: 0.9950
Average F1 Score: 0.6948
